<a href="https://colab.research.google.com/github/itsmerajesh4990/AIpracticeandtraining/blob/main/crewai_multiagent_level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gitex workshop: Create Multi Agents to Research and Write an Article

In this workshop, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

https://huggingface.co/spaces/decodingdatascience/fromprompttopost

In [23]:
#install packages
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [24]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [25]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.



In [26]:
#setting the model
import os

#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-5.2-2025-12-11'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano-2025-04-14'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [27]:
# Retrieve API key from Colab secrets and set it in environment variables
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [28]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [29]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = userdata.get('serperapi') ##"218873db9b73d345d3083e2f1f29cb74b14d4ddb" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [30]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [31]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [32]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [33]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [34]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.2500 words",
    agent=writer,
)

### Task: Edit

In [35]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [36]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=4
)

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

## Running the Crew
## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [37]:
topic = "GPT 5.2"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on GPT 5.2.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...


Thought: To ensure I gather the most relevant and up-to-date information about GPT 5.2, I will first search the internet for recent news, trends, and key updates related to GPT 5.2. This will help identify the latest developments, key players, and noteworthy news. After that, I will analyze the target audience for the article, considering their interests and pain points. Then, I will develop a detailed content outline, including an introduction, key points, and a call to action, along with SEO keywords and relevant sources. 

Action: Search the internet
Action Input: {"search_query": "GPT 5.2 latest news, updates, key features, trends 2024"}


 


Search results: Title: Introducing GPT-5.2 - OpenAI
Link: https://openai.com/index/introducing-gpt-5-2/
Snippet: Overall, GPT‑5.2 brings significant improvements in general intelligence, long-context understanding, agentic tool-calling, and vision—making it ...
---
Title: OpenAI's GPT-5.2 is here: what enterprises need to know
Link: https://venturebeat.com/ai/openais-gpt-5-2-is-here-what-enterprises-need-to-know
Snippet: "We designed 5.2 to unlock even more economic value for people. It's better at creating spreadsheets, building presentations, writing code, ...
---
Title: OpenAI calls GPT-5.2 its most advanced model for work
Link: https://www.constellationr.com/blog-news/insights/openai-calls-gpt-52-its-most-advanced-model-work
Snippet: "We designed GPT‑5.2 to unlock even more economic value for people; it's better at creating spreadsheets, building presentations, writing code, ...
---
Title: GPT 5.2 and useful patterns for building HTML tools
Link: https://simonw.substack.com/p/g

Thought: I have gathered recent and relevant information about GPT 5.2 from multiple sources, including OpenAI's official announcement, industry analyses, and expert discussions. This provides a solid foundation for understanding the latest trends, key features, and noteworthy news about GPT 5.2. Next, I will analyze the target audience for the article—likely AI enthusiasts, developers, enterprise decision-makers, and tech strategists—and consider their interests and pain points. Based on this, I will develop a detailed content outline, highlighting key points such as improvements, practical applications, and strategic implications. I will also identify SEO keywords to optimize visibility and include references to authoritative sources for credibility.

Action: List files in directory()
Action Input: {} 

Action 'List files in directory()' don't exist, these are the only available Actions: Search the internet: Search the internet(search_query: 'string') - A tool that can be used to sem

Thought: I have gathered substantial recent information about GPT 5.2 from multiple credible sources, including OpenAI's official announcements, industry analysis, and expert commentary. This provides insight into its key features, improvements, and strategic implications. Now, I need to analyze the target audience, which likely includes AI enthusiasts, developers, enterprise decision-makers, and tech strategists, each with specific interests and pain points such as understanding the latest capabilities, practical applications, and strategic advantages. 

Next, I will develop a detailed content plan that includes an engaging outline, targeted SEO keywords, and references to authoritative sources to ensure the article is both informative and optimized for search engines. This comprehensive plan will serve as the blueprint for the Content Writer to craft a compelling, factually accurate, and audience-focused article.

Final Answer:

---

**Comprehensive Content Plan for Blog Article on G

Thought: I now can give a great answer

Final Answer:

```markdown
# Unlocking the Future with GPT 5.2: The Latest Breakthrough in AI Technology

## Introduction: The Evolution of GPT and the Significance of GPT 5.2

Since its inception, the Generative Pre-trained Transformer (GPT) series by OpenAI has revolutionized the landscape of artificial intelligence, transforming how machines understand and generate human-like language. Starting from GPT-1, each iteration has brought significant leaps in capability, culminating in GPT 4, which set new standards for natural language understanding, contextual awareness, and versatility. Now, with the release of GPT 5.2, the AI community stands poised on the brink of a new era—one characterized by enhanced intelligence, more sophisticated reasoning, and broader multimodal capabilities.

The importance of GPT 5.2 in today’s AI landscape cannot be overstated. Industry analysts highlight that this version not only refines previous strengths but intro

In [38]:
from IPython.display import Markdown
Markdown(result)

```markdown
# Unlocking the Future with GPT 5.2: The Latest Breakthrough in AI Technology

## Introduction: The Evolution of GPT and the Significance of GPT 5.2

Since its inception, the Generative Pre-trained Transformer (GPT) series by OpenAI has significantly advanced the field of artificial intelligence, shaping how machines understand and generate human-like language. From GPT-1 to GPT 4, each iteration has brought notable improvements in natural language processing, contextual comprehension, and versatility. The recent release of GPT 5.2 marks a new milestone, promising enhanced capabilities that could influence various sectors and applications.

GPT 5.2's emergence underscores the ongoing progress in AI technology, with industry experts highlighting its potential to improve enterprise workflows, automation, and human-computer interaction. As AI continues to expand its reach across industries such as healthcare, finance, and creative content, GPT 5.2 offers promising tools for increased efficiency and innovation. This article explores the technical advancements, practical applications, and strategic implications of GPT 5.2, providing insights into its role in shaping the future of AI.

## What is GPT 5.2? An Overview

OpenAI announced GPT 5.2 in early 2024 as a significant evolution within the GPT series. Building upon GPT 4's foundation, GPT 5.2 introduces core enhancements aimed at improving general intelligence, understanding of context, and multimodal processing. According to OpenAI and industry sources, these updates address previous limitations and expand the model’s capabilities.

A key feature of GPT 5.2 is its improved ability to process and understand long-context data. Unlike earlier versions, which struggled with extended conversations or complex documents, GPT 5.2 can maintain coherence over longer interactions. It also demonstrates enhanced reasoning skills, enabling it to perform more sophisticated decision-making tasks. Additionally, GPT 5.2 integrates advanced visual processing, allowing it to interpret images, videos, and other visual data—marking a step toward truly multimodal AI systems.

Performance improvements are also notable. OpenAI reports that GPT 5.2 achieves state-of-the-art results across various benchmarks, with faster inference speeds and increased accuracy. These advancements make GPT 5.2 more practical for real-world use, especially in enterprise settings where reliability and efficiency are critical.

## Key Features and Innovations

### Enhanced Long-Context Understanding

One of GPT 5.2’s most significant features is its ability to comprehend and utilize long-context information more effectively. This allows the model to process entire documents, lengthy conversations, or complex datasets without losing relevance or coherence. Such capabilities are valuable in fields like legal, academic, and technical writing, where detailed analysis and context-aware responses are essential.

### Improved Reasoning and Decision-Making

GPT 5.2 shows marked improvements in logical reasoning, problem-solving, and decision-making. These are achieved through refined training techniques and architectural adjustments that bolster its cognitive functions. As a result, the model can handle complex tasks such as strategic planning and troubleshooting, making it an asset for enterprise applications requiring accuracy and nuance.

### Advanced Vision Processing

A notable leap in GPT 5.2 is its integrated visual understanding. The model can interpret images, videos, and other visual inputs, enabling functionalities like visual data analysis, image captioning, and multimodal content creation. This opens new opportunities in sectors such as healthcare diagnostics, manufacturing, and digital media.

### Agentic Tool-Calling and Automation

GPT 5.2’s ability to call external tools and APIs autonomously enhances its role as an automation agent. It can execute functions such as data retrieval, software control, or workflow initiation without human intervention. This capability is a step toward more autonomous AI systems capable of managing complex, multi-step processes.

### Performance Metrics and Benchmarks

OpenAI reports that GPT 5.2 has achieved leading positions across various AI benchmarks, including natural language understanding and multimodal tasks. Its efficiency gains also result in reduced computational costs and faster response times, making it more accessible for commercial deployment. These metrics emphasize GPT 5.2’s role as a state-of-the-art model setting new standards in AI performance.

## Practical Applications and Use Cases

### Enterprise Productivity and Automation

GPT 5.2’s advanced features are transforming enterprise workflows. Organizations can automate routine tasks such as report generation, data analysis, and content creation, saving time and resources. For example, financial institutions can utilize GPT 5.2 to analyze market data, generate summaries, and support strategic decisions, enhancing overall efficiency.

### Content Creation and Editing

Content creators benefit from GPT 5.2’s sophisticated language generation and editing tools. It can assist in drafting articles, marketing content, or scripts, ensuring high-quality output with minimal human input. Its multimodal capabilities also facilitate the creation of visual content, infographics, and multimedia presentations, streamlining creative workflows.

### Customer Support Automation

Many organizations are deploying GPT 5.2 to improve customer service. Its enhanced understanding allows for more accurate and empathetic responses, reducing the need for human intervention in routine inquiries. The model’s ability to interpret visual data also enables support systems to handle images or videos sent by customers, enriching the service experience.

### Data Analysis and Decision Support

GPT 5.2’s reasoning skills make it a valuable tool for analyzing complex data sets. It can generate summaries, identify insights, and support decision-making processes across sectors like healthcare, finance, and research. For example, it can assist in diagnosing medical images and correlating patient data to suggest potential treatments.

### Developer Tools and Custom Solutions

OpenAI’s API for GPT 5.2 provides developers with versatile tools to build tailored AI applications. From intelligent chatbots to automation pipelines, the model’s flexible architecture supports diverse industry needs. Its ability to call external APIs further enhances integration, enabling organizations to develop bespoke solutions.

## Strategic Implications for Businesses

### Gaining a Competitive Edge

Adopting GPT 5.2 early can provide organizations with a strategic advantage. Its advanced capabilities enable more efficient operations, innovative product development, and improved customer engagement. Companies that leverage these features can differentiate themselves in competitive markets.

### Cost Savings and Productivity Gains

GPT 5.2’s automation and accuracy can lead to significant cost reductions. Automating routine tasks allows human resources to focus on higher-value activities, while the model’s precision minimizes errors. Over time, these efficiencies can enhance profitability and accelerate project timelines.

### Future-Proofing AI Strategies

Investing in GPT 5.2 positions organizations to adapt to ongoing AI advancements. Early adoption facilitates integration with future models and features, ensuring that companies remain at the forefront of technological innovation. This proactive approach supports long-term strategic planning.

### Ethical and Responsible AI Use

While GPT 5.2 offers impressive functionalities, responsible deployment is essential. Organizations should prioritize ethical considerations such as bias mitigation, data privacy, and transparency. Establishing governance frameworks ensures that AI benefits are maximized while risks are minimized.

## Challenges and Considerations

### Integration Complexity

Incorporating GPT 5.2 into existing systems may require significant technical effort. Organizations need to ensure infrastructure readiness, staff training, and possible customization to optimize its use. Careful planning and skilled teams are vital for seamless integration.

### Data Privacy and Security

Handling sensitive information with GPT 5.2 necessitates stringent security protocols. Companies must adhere to data privacy regulations and implement measures to prevent breaches or misuse. Ensuring data integrity is critical, especially when processing confidential information.

### Fairness and Bias Mitigation

Despite technological advancements, AI models can still reflect biases present in training data. Continuous monitoring and mitigation strategies are essential to prevent biased outcomes, particularly in customer-facing or high-stakes applications.

### Performance and Reliability Expectations

While GPT 5.2 represents the latest in AI development, users should maintain realistic expectations regarding its reliability. Regular oversight, updates, and human judgment remain important to ensure optimal performance and accuracy.

## The Future of GPT and AI Development

GPT 5.2 signifies a significant step toward more advanced AI systems, with ongoing developments aimed at achieving greater autonomy, safety, and multimodal integration. Future models are expected to become increasingly adaptable and human-like in their reasoning and interactions. OpenAI's future roadmap indicates continued focus on safety measures and broader capabilities.

As AI tools evolve, collaboration among technologists, policymakers, and industry leaders will be crucial to ensure responsible and ethical advancements. The integration of GPT 5.2 into various sectors signals a future where human-AI collaboration becomes more seamless, productive, and impactful.

## Conclusion: Embracing the AI Future with GPT 5.2

GPT 5.2 represents a notable leap forward in artificial intelligence, combining sophisticated language understanding, multimodal processing, and automation features. Its innovations hold the promise of transforming productivity, creativity, and strategic decision-making across industries. For organizations and developers aiming to stay competitive, adopting GPT 5.2 offers a valuable opportunity to innovate, optimize operations, and prepare for future AI advancements.

However, harnessing its full potential requires careful planning around integration, ethical considerations, and security. A balanced approach—grounded in responsible use and strategic foresight—will be essential to maximize benefits while mitigating risks. As AI technology continues to evolve rapidly, understanding and responsibly deploying GPT 5.2 will be key to shaping a more efficient and equitable digital future.

---

*Interested in exploring GPT 5.2? Visit [OpenAI’s official page](https://openai.com/index/introducing-gpt-5-2/) to learn more, and stay tuned for updates on how this cutting-edge technology will redefine AI in 2024 and beyond.*

---

**Keywords:** GPT 5.2, OpenAI GPT updates 2024, AI language models, GPT 5.2 features, enterprise AI solutions, AI automation tools, AI in business, long-context understanding AI, AI vision capabilities
```

# Deploy this crew in gradio

In [39]:
pip install gradio

  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
Using cached typer-0.20.0-py3-none-any.whl (47 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.20.0 which is incompatible.


In [40]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1500 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="AI Agent: Prompt to Post",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3330e49b7052327ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
